In [ ]:
#Scraping xe.gr
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from zenrows import ZenRowsClient
from IPython.display import display, HTML


In [13]:

# RENTALS 
# Set the user-agent header
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# Create a proxy client
client = ZenRowsClient("a45285ef2974d1cf4c9f103296a7835b19844643")
BASE_URL='https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic'
response = client.get(BASE_URL, headers=headers)
soup=BeautifulSoup(response.content,"html.parser")

# PAGINATION : Finding all li tags in ul and printing the text within it 
web_urls=[]
nums=[]

buttons=soup.find('ul',class_='results-pagination')
for li in buttons.find_all('li'):
    nums.append(li.text.replace('\n',''))

# Take the last element from the list of texts found in li elements
last_page=int(nums[-1])
print(last_page*34)


#Create the urls of multiple "xe" result pages, 
for i in range(1,last_page+1):
    web_url_base="https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page="
    web_urls.append(web_url_base+str(i))
    
print(web_urls)



4590
['https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=1', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=2', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=3', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=4', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=5', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=6', 'https://www.xe.gr/en/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=7', 'https:

In [39]:
#SALES

# Create a proxy client
BASE_URL_Sales='https://www.xe.gr/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic'
response_Sales = client.get(BASE_URL_Sales, headers=headers)
soup_Sales=BeautifulSoup(response_Sales.content,"html.parser")

# PAGINATION : Finding all li tags in ul and printing the text within it 
num_Sales=[]
web_urls_Sales=[]
buttons_Sales=soup_Sales.find('ul',class_='results-pagination')
for li in buttons_Sales.find_all('li'):
    num_Sales.append(li.text.replace('\n',''))

# Take the last element from the list of texts found in li elements
last_page_Sales=int(num_Sales[-1])
print(last_page_Sales)

num_of_ads_sales=last_page_Sales*34
print(num_of_ads_sales)


for i in range(1,last_page_Sales+1):
    web_url_Sales_base="https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page="
    web_urls_Sales.append(web_url_Sales_base+str(i))
    
    
print(web_urls_Sales)



295
10030
['https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=1', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=2', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=3', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=4', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=5', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=6', 'https://www.xe.gr/en/property/results?transaction_name=buy&item_type=re_residence&geo_place_ids%5B%5D=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page=7', 'https://

In [38]:
# RENTALS PAGES
# Access page source code, parse, find elements and write them in csv file 
price_list=[]
title_list=[]
level_list=[]
url_list=[]
address_list=[]
price_sqm_list=[]
    


for web_url in web_urls:
    try:
        response=client.get(web_url,headers=headers)
    
        xe_webpage=response.text
    
        soup=BeautifulSoup(xe_webpage,"html.parser")
        print('OK')
    except:
        print(f"No response for {web_url}")

    
    
    prices=soup.find_all(name="span",class_="property-ad-price")
    titles=soup.find_all(name="div",class_="common-property-ad-title")
    levels=soup.find_all(name="span",class_="property-ad-level")
    url_element=soup.find_all(name="div",class_="common-property-ad-body grid-y align-justify")
    addresses=soup.find_all(name="span",class_="common-property-ad-address")
    prices_sqm=soup.find_all(name="span",class_="property-ad-price-per-sqm")
    
    for price in prices:
        p=re.findall(r'\d+', price.getText())
        price_list.append(int("".join(p)))
       #price_list.append(price.getText().replace("\xa0"," ")) 
    
    for title in titles:
        t=re.findall(r'\d+', title.getText())
        title_list.append(int("".join(t)))
        #title_list.append(title.getText().replace("\n",""))
    
    for level in levels:
        y1=level.getText().replace("\n","")
        level_list.append(y1.replace(" ", ""))
        
    for urlz in url_element:
        urls=urlz.find('a')
        url_list.append(urls.get('href'))
    
    for address in addresses:
        address_clean=re.findall(r'\((.*?)\)',address.getText())
        try:
            address_list.append(address_clean[0])
        except:
            address_list.append('NA')
        #address_clean=address.getText().replace(" ", "")
        #address_list.append(address_clean.replace("\n",""))  
    
    for price_sqm in prices_sqm:
        psqm=re.findall(r'\d+', price_sqm.getText())
        price_sqm_list.append(int("".join(psqm)))
        #price_sqm_list.append((price_sqm.getText().replace("\xa0","")).replace("\n",""))
    
#print(price_list)
#print(title_list)
#print(level_list)
#print(url_list)
#print(address_list)
#print(price_sqm_list)
# print(len(price_list))
# print(len(title_list))
# print(len(level_list))
# print(len(url_list))
# print(len(address_list))
# print(len(price_sqm_list))

    
    
    
#Create dictionary with column names as keys
dict={'Sqm':title_list,'Price':price_list,'URL':url_list, 'neighbourhood':address_list, 'price_sq/m':price_sqm_list}
df=pd.DataFrame(dict)
#create dataframe
#df=pd.DataFrame(list(zip(price_list,title_list,url_list,address_list,price_sqm_list)), columns=['price','title','url','address','price_sqm'

#duplicateRows = df[df.duplicated(['Title', 'Price','neighbourhood'])]
#write to csv file
df.to_csv(f'xe_rentals_{date.today()}',mode='a',index=False, header=False)

time.sleep(1)


OK
OK
OK
102
102
104
102
102
102


In [36]:
display(df)

,Sqm,Price,URL,neighbourhood,price_sq/m
0,30,450,https://www.xe.gr/en/property/d/property-to-re...,Gyzi,15
1,81,460,https://www.xe.gr/en/property/d/property-to-re...,Ano Patisia,6
2,124,2700,https://www.xe.gr/en/property/d/property-to-re...,Kolonaki,22
3,69,650,https://www.xe.gr/en/property/d/property-to-re...,Ano Petralona,9
4,67,800,https://www.xe.gr/en/property/d/property-to-re...,Strefi Hill,12
...,...,...,...,...,...
4517,800,5000,https://www.xe.gr/en/property/d/property-to-re...,Omonia,6
4518,65,900,https://www.xe.gr/en/property/d/property-to-re...,Ano Ilisia,14
4519,35,650,https://www.xe.gr/en/property/d/property-to-re...,NA,19
4520,75,650,https://www.xe.gr/en/property/d/property-to-re...,NA,9


In [68]:
price_list=[]
title_list=[]
level_list=[]
url_list=[]
address_list=[]
price_sqm_list=[]
constru_year_list=[]


for web_url in web_urls_Sales:
    try:
        response=client.get(web_url,headers=headers)
    
        xe_webpage=response.text
    
        soup=BeautifulSoup(xe_webpage,"html.parser")
        print('OK')
    except:
        print(f"No response for {web_url}")

    
    
    prices=soup.find_all(name="span",class_="property-ad-price")
    titles=soup.find_all(name="div",class_="common-property-ad-title")
    levels=soup.find_all(name="span",class_="property-ad-level")
    url_element=soup.find_all(name="div",class_="common-property-ad-body grid-y align-justify")
    addresses=soup.find_all(name="span",class_="common-property-ad-address")
    prices_sqm=soup.find_all(name="span",class_="property-ad-price-per-sqm")
    #construction_dates=soup.find_all(name="div",class_="grid-x property-ad-construction-year-container")

    
    for price in prices:
        p=re.findall(r'\d+', price.getText())
        price_list.append(int("".join(p)))
       #price_list.append(price.getText().replace("\xa0"," ")) 
    
    for title in titles:
        t=re.findall(r'\d+', title.getText())
        title_list.append(int("".join(t)))
        #title_list.append(title.getText().replace("\n",""))
    
    for level in levels:
        y1=level.getText().replace("\n","")
        level_list.append(y1.replace(" ", ""))
        
    for urlz in url_element:
        urls=urlz.find('a')
        url_list.append(urls.get('href'))
    
    for address in addresses:
        address_clean=re.findall(r'\((.*?)\)',address.getText())
        try:
            address_list.append(address_clean[0])
        except:
            address_list.append('NA')
        #address_clean=address.getText().replace(" ", "")
        #address_list.append(address_clean.replace("\n",""))  
    
    for price_sqm in prices_sqm:
        psqm=re.findall(r'\d+', price_sqm.getText())
        price_sqm_list.append(int("".join(psqm)))
        #price_sqm_list.append((price_sqm.getText().replace("\xa0","")).replace("\n",""))

    # for constru_date in construction_dates:
    #     try:
    #         year=re.findall(r'\d+',constru_date.getText())
    #         constru_year_list.append(int("".join(year)))
    #     except:
    #         constru_year_list.append("None")
    
print(len(price_list))
print(len(title_list))
print(len(level_list))
print(len(url_list))
print(len(address_list))
print(len(price_sqm_list))

    
    
    
#Create dictionary with column names as keys
dict={'Sqm':title_list,'Price':price_list,'URL':url_list, 'neighbourhood':address_list, 'price_sq/m':price_sqm_list}

#create dataframe
df=pd.DataFrame(dict)
#df=pd.DataFrame(list(zip(price_list,title_list,url_list,address_list,price_sqm_list)), columns=['price','title','url','address','price_sqm'
#duplicateRows = df[df.duplicated(['Title', 'Price','neighbourhood'])]


#write to csv file
df.to_csv(f'xe_Sales_{date.today()}',mode='a',index=False, header=False)

time.sleep(1)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
10000
10000
11340
10000
10000
10000
0


In [69]:
display(df)

,Sqm,Price,URL,neighbourhood,price_sq/m
0,85,240000,https://www.xe.gr/en/property/d/property-for-s...,Gyzi,2824
1,71,160000,https://www.xe.gr/en/property/d/property-for-s...,Gyzi,2254
2,50,125000,https://www.xe.gr/en/property/d/property-for-s...,Agios Loukas,2500
3,76,265000,https://www.xe.gr/en/property/d/property-for-s...,Tris Gefires,3487
4,97,285000,https://www.xe.gr/en/property/d/property-for-s...,Pagkrati,2938
...,...,...,...,...,...
9995,92,120000,https://www.xe.gr/en/property/d/property-for-s...,Nirvana,1304
9996,97,108000,https://www.xe.gr/en/property/d/property-for-s...,Agios Loukas,1113
9997,73,140000,https://www.xe.gr/en/property/d/property-for-s...,Pedion tou Areos,1918
9998,46,80000,https://www.xe.gr/en/property/d/property-for-s...,Neapoli,1739


In [64]:
print(len(construction_dates))
print(len(title_list))

27
68
